In [1]:
import numpy as np
import random
import time
from math import *


######################################################################
# This section contains functions for loading CSV (comma separated values)
# files and convert them to a dataset of instances.
# Each instance is a tuple of attributes. The entire dataset is a list
# of tuples.
######################################################################

# Loads a CSV files into a list of tuples.
# Ignores the first row of the file (header).
# Numeric attributes are converted to floats, nominal attributes
# are represented with strings.
# Parameters:
#   fileName: name of the CSV file to be read
# Returns: a list of tuples
def loadCSV(fileName):
    fileHandler = open(fileName, "rt")
    lines = fileHandler.readlines()
    fileHandler.close()
    del lines[0] # remove the header
    dataset = []
    for line in lines:
        instance = lineToTuple(line)
        dataset.append(instance)
    # print(dataset)
    return dataset

# Converts a comma separated string into a tuple
# Parameters
#   line: a string
# Returns: a tuple
def lineToTuple(line):
    # remove leading/trailing witespace and newlines
    cleanLine = line.strip()
    # get rid of quotes
    cleanLine = cleanLine.replace('"', '')
    # separate the fields
    lineList = cleanLine.split(",")
    # convert strings into numbers
    stringsToNumbers(lineList)
    lineTuple = tuple(lineList)
    
    return lineTuple

# Destructively converts all the string elements representing numbers
# to floating point numbers.
# Parameters:
#   myList: a list of strings
# Returns None
def stringsToNumbers(myList):
    for i in range(len(myList)):
        if (isValidNumberString(myList[i])):
            myList[i] = float(myList[i])

# Checks if a given string can be safely converted into a positive float.
# Parameters:
#   s: the string to be checked
# Returns: True if the string represents a positive float, False otherwise
def isValidNumberString(s):
  if len(s) == 0:
    return False
  if  len(s) > 1 and s[0] == "-":
      s = s[1:]
  for c in s:
    if c not in "0123456789.":
      return False
  return True


######################################################################
# This section contains functions for clustering a dataset
# using the k-means algorithm.
######################################################################
def distance(instance1, instance2):
    if instance1 == None or instance2 == None:
        return float("inf")
    sumOfSquares = 0
    for i in range(1, len(instance1)):
        sumOfSquares += (instance1[i] - instance2[i])**2
    return sqrt(sumOfSquares)

def meanInstance(name, instanceList):
    numInstances = len(instanceList)
    if (numInstances == 0):
        return
    numAttributes = len(instanceList[0])
    means = [name] + [0] * (numAttributes-1)
    for instance in instanceList:
        for i in range(1, numAttributes):
            means[i] += instance[i]
    for i in range(1, numAttributes):
        means[i] /= float(numInstances)
    return tuple(means)

def assign(instance, centroids):
    minDistance = distance(instance, centroids[0])
    minDistanceIndex = 0
    for i in range(1, len(centroids)):
        d = distance(instance, centroids[i])
        if (d < minDistance):
            minDistance = d
            minDistanceIndex = i
    return minDistanceIndex

def createEmptyListOfLists(numSubLists):
    myList = []
    for i in range(numSubLists):
        myList.append([])
    return myList

def assignAll(instances, centroids):
    clusters = createEmptyListOfLists(len(centroids))
    for instance in instances:
        clusterIndex = assign(instance, centroids)
        clusters[clusterIndex].append(instance)
    return clusters

def computeCentroids(clusters):
    centroids = []
    for i in range(len(clusters)):
        name = "\nCentroid " + str(i+1)
        centroid = meanInstance(name, clusters[i])
        centroids.append(centroid)
    return centroids

def kmeans(instances, k, animation=False, initCentroids=None):
    result = {}
    if (initCentroids == None or len(initCentroids) < k):
        # randomly select k initial centroids
       
        centroids = random.sample(instances, k)
    else:
        centroids = initCentroids
    prevCentroids = []

    iteration = 0
    while (centroids != prevCentroids):
        iteration += 1
        clusters = assignAll(instances, centroids)
        prevCentroids = centroids
        centroids = computeCentroids(clusters)
        withinss = computeWithinss(clusters, centroids)
        # sse =computeWithinss(clusters, centroids)
        
    result["clusters"] = clusters
    result["centroids"] = centroids
    result["withinss"] = withinss
    result["sse"]=withinss
    print("iterations", iteration)
    return result

def computeWithinss(clusters, centroids):
    result = 0
    for i in range(len(centroids)):
        centroid = centroids[i]
        cluster = clusters[i]
        for instance in cluster:
            result += distance(centroid, instance)
    return result

# Repeats k-means clustering n times, and returns the clustering
# with the smallest withinss
def repeatedKMeans(instances, k, n):
    bestClustering = {}
    bestClustering["withinss"] = float("inf")
    for i in range(1, n+1):
        print ("k-means trial %d," % i)
        trialClustering = kmeans(instances, k)
        print(trialClustering)
        print ("withinss: %.1f" % trialClustering["withinss"])
        if trialClustering["withinss"] < bestClustering["withinss"]:
            bestClustering = trialClustering
            minWithinssTrial = i
    print ("Trial with minimum withinss:", minWithinssTrial)
    return bestClustering

def printTable(instances):
    # print(instances)
    for instance in instances:
        if instance != None:
            line = instance[0] + "\t"
            for i in range(1, len(instance)):
                line += "%.2f " % instance[i]
            print (line)

In [2]:
######################################################################
# Test code - Euclidean
######################################################################

dataset = loadCSV(r"C:\Users\cinti\Downloads\iris_original.csv")
clustering = kmeans(dataset, 3)
printTable(clustering["centroids"])

def printT(i):
    print("\nSum of Squared Error is: "+str(i))
printT(clustering["sse"])

a=clustering["sse"]
import csv
with open("Kmeans_Euclidean_SSE.csv", "w") as fp_out:
    print(clustering["sse"],file=fp_out)

clust = repeatedKMeans(dataset, 3, 100)

iterations 7

Centroid 1	2.76 4.35 1.39 

Centroid 2	3.03 5.70 2.09 

Centroid 3	3.42 1.46 0.24 

Sum of Squared Error is: 74.26879704399985
k-means trial 1,
iterations 7
{'clusters': [[(5.1, 3.5, 1.4, 0.2), (4.9, 3.0, 1.4, 0.2), (4.7, 3.2, 1.3, 0.2), (4.6, 3.1, 1.5, 0.2), (5.0, 3.6, 1.4, 0.2), (5.4, 3.9, 1.7, 0.4), (4.6, 3.4, 1.4, 0.3), (5.0, 3.4, 1.5, 0.2), (4.4, 2.9, 1.4, 0.2), (4.9, 3.1, 1.5, 0.1), (5.4, 3.7, 1.5, 0.2), (4.8, 3.4, 1.6, 0.2), (4.8, 3.0, 1.4, 0.1), (4.3, 3.0, 1.1, 0.1), (5.8, 4.0, 1.2, 0.2), (5.7, 4.4, 1.5, 0.4), (5.4, 3.9, 1.3, 0.4), (5.1, 3.5, 1.4, 0.3), (5.7, 3.8, 1.7, 0.3), (5.1, 3.8, 1.5, 0.3), (5.4, 3.4, 1.7, 0.2), (5.1, 3.7, 1.5, 0.4), (4.6, 3.6, 1.0, 0.2), (5.1, 3.3, 1.7, 0.5), (4.8, 3.4, 1.9, 0.2), (5.0, 3.0, 1.6, 0.2), (5.0, 3.4, 1.6, 0.4), (5.2, 3.5, 1.5, 0.2), (5.2, 3.4, 1.4, 0.2), (4.7, 3.2, 1.6, 0.2), (4.8, 3.1, 1.6, 0.2), (5.4, 3.4, 1.5, 0.4), (5.2, 4.1, 1.5, 0.1), (5.5, 4.2, 1.4, 0.2), (4.9, 3.1, 1.5, 0.1), (5.0, 3.2, 1.2, 0.2), (5.5, 3.5, 1.3, 0.2), 

In [3]:
# Jaccard Distance 


def jdistance(a,b):
    c = set(a).intersection(set(b))
    return (1- float(len(c)) / (len(a) + len(b) - len(c)))

def computeWithinss(clusters, centroids):
    result = 0
    for i in range(len(centroids)):
        centroid = centroids[i]
        cluster = clusters[i]
        for instance in cluster:
            result += jdistance(centroid, instance)
    return result

# Repeats k-means clustering n times, and returns the clustering
# with the smallest withinss
def repeatedKMeans(instances, k, n):
    bestClustering = {}
    bestClustering["withinss"] = float("inf")
    for i in range(1, n+1):
        print ("k-means trial %d," % i)
        trialClustering = kmeans(instances, k)
        print(trialClustering)
        print ("withinss: %.1f" % trialClustering["withinss"])
        if trialClustering["withinss"] < bestClustering["withinss"]:
            bestClustering = trialClustering
            minWithinssTrial = i
    print ("Trial with minimum withinss:", minWithinssTrial)
    return bestClustering


def printTable(instances):
    # print(instances)
    for instance in instances:
        if instance != None:
            line = instance[0] + "\t"
            for i in range(1, len(instance)):
                line += "%.2f " % instance[i]
            print (line)

def extractAttribute(instances, index):
    result = []
    for instance in instances:
        result.append(instance[index])
    return result

######################################################################
# Test code - Jaccard Distance 
######################################################################

dataset = loadCSV(r"C:\Users\cinti\Downloads\iris_original.csv")
clustering = kmeans(dataset, 3)
printTable(clustering["centroids"])

# printT(clustering["clusters"])
def printT(i):
    print("\nSum of Squared Error is: "+str(i))
# printT(clustering["clusters"])
printT(clustering["sse"])
# printTable(clustering["clusters"])
a=clustering["sse"]


import csv
with open("Kmeans_Jaccard_SSE.csv", "w") as fp_out:
    print(clustering["sse"],file=fp_out)

clust = repeatedKMeans(dataset, 3, 100)

iterations 7

Centroid 1	3.03 5.70 2.09 

Centroid 2	3.42 1.46 0.24 

Centroid 3	2.76 4.35 1.39 

Sum of Squared Error is: 150.0
k-means trial 1,
iterations 3
{'clusters': [[(7.0, 3.2, 4.7, 1.4), (6.4, 3.2, 4.5, 1.5), (6.9, 3.1, 4.9, 1.5), (5.5, 2.3, 4.0, 1.3), (6.5, 2.8, 4.6, 1.5), (5.7, 2.8, 4.5, 1.3), (6.3, 3.3, 4.7, 1.6), (4.9, 2.4, 3.3, 1.0), (6.6, 2.9, 4.6, 1.3), (5.2, 2.7, 3.9, 1.4), (5.0, 2.0, 3.5, 1.0), (5.9, 3.0, 4.2, 1.5), (6.0, 2.2, 4.0, 1.0), (6.1, 2.9, 4.7, 1.4), (5.6, 2.9, 3.6, 1.3), (6.7, 3.1, 4.4, 1.4), (5.6, 3.0, 4.5, 1.5), (5.8, 2.7, 4.1, 1.0), (6.2, 2.2, 4.5, 1.5), (5.6, 2.5, 3.9, 1.1), (5.9, 3.2, 4.8, 1.8), (6.1, 2.8, 4.0, 1.3), (6.3, 2.5, 4.9, 1.5), (6.1, 2.8, 4.7, 1.2), (6.4, 2.9, 4.3, 1.3), (6.6, 3.0, 4.4, 1.4), (6.8, 2.8, 4.8, 1.4), (6.7, 3.0, 5.0, 1.7), (6.0, 2.9, 4.5, 1.5), (5.7, 2.6, 3.5, 1.0), (5.5, 2.4, 3.8, 1.1), (5.5, 2.4, 3.7, 1.0), (5.8, 2.7, 3.9, 1.2), (6.0, 2.7, 5.1, 1.6), (5.4, 3.0, 4.5, 1.5), (6.0, 3.4, 4.5, 1.6), (6.7, 3.1, 4.7, 1.5), (6.3, 2.3, 4

In [5]:
# Cosine distance


def distance(instance1, instance2):
    if instance1 == None or instance2 == None:
        return float("inf")
    sumOfSquares = 0
    for i in range(1, len(instance1)):
        sumOfSquares += (instance1[i] - instance2[i])**2
    return sumOfSquares


#Calculate cosine distance
def square_rooted(x):
    return round(sqrt(sum([a*a for a in x])),3)
def cdistance(x,y):
    numerator = sum([a*b for a,b in zip(x,list(y))])
    denominator = square_rooted(x)*square_rooted(y)
    return (1-round(numerator/float(denominator),3))

def meanInstance(name, instanceList):
    numInstances = len(instanceList)
    if (numInstances == 0):
        return
    numAttributes = len(instanceList[0])
    means = [name] + [0] * (numAttributes-1)
    for instance in instanceList:
        for i in range(1, numAttributes):
            means[i] += instance[i]
    for i in range(1, numAttributes):
        means[i] /= float(numInstances)
    return tuple(means)

def assign(instance, centroids):
    minDistance = distance(instance, centroids[0])
    # print ('centroid index: 0')
    minDistanceIndex = 0
    for i in range(1, len(centroids)):
        d = distance(instance, centroids[i])
        #print ('centroid index: {}'.format(i))
        #print(centroids[i])
        if (d < minDistance):
            minDistance = d
            minDistanceIndex = i
    
    return minDistanceIndex

def createEmptyListOfLists(numSubLists):
    myList = []
    for i in range(numSubLists):
        myList.append([])
    return myList

def assignAll(instances, centroids):
    clusters = createEmptyListOfLists(len(centroids))
    for instance in instances:
        clusterIndex = assign(instance, centroids)
        clusters[clusterIndex].append(instance)
    print(clusters[1])
    return clusters

def computeCentroids(clusters):
    centroids = []
    for i in range(len(clusters)):
        # name = "centroid" + str(i)
        name = i
        centroid = meanInstance(name, clusters[i])
        centroids.append(centroid)
    return centroids

def kmeans(instances, k, animation=False, initCentroids=None):
    result = {}
    if (initCentroids == None or len(initCentroids) < k):
        # randomly select k initial centroids
        centroids = random.sample(instances, k)
        # print ()"Printing centroid"
        # print centroids
    else:
        centroids = initCentroids
    prevCentroids = []

    iteration = 0
    while (centroids != prevCentroids):
        iteration += 1
        # print '#iter: {}'.format(iteration)
        clusters = assignAll(instances, centroids)

        prevCentroids = centroids
        # print centroids
        centroids = computeCentroids(clusters)

        withinss = computeWithinss(clusters, centroids)

    result["clusters"] = clusters
    result["centroids"] = centroids
    result["withinss"] = withinss
    print("iteration", iteration)
    return result

def computeWithinss(clusters, centroids):
    result = 0
    for i in range(len(centroids)):
        centroid = centroids[i]
        cluster = clusters[i]
        for instance in cluster:
            result += cdistance(centroid, instance)
    return result

# Repeats k-means clustering n times, and returns the clustering
# with the smallest withinss
def repeatedKMeans(instances, k, n):
    bestClustering = {}
    bestClustering["withinss"] = float("inf")
    for i in range(1, n+1):
        print ("k-means trial %d," % i) 
        trialClustering = kmeans(instances, k)
        print ("withinss: %.1f" % trialClustering["withinss"])
        if trialClustering["withinss"] < bestClustering["withinss"]:
            bestClustering = trialClustering
            minWithinssTrial = i
    print ("Trial with minimum withinss:", minWithinssTrial)
    return bestClustering


######################################################################
# This section contains functions for visualizing datasets and
# clustered datasets.
######################################################################

def printTable(instances):
    for instance in instances:
        if instance != None:
            # line = instance[0] + "\t"
            line = instance[0]
            # print instance
            # print len(instance)
            for i in range(1, len(instance)):
                # print instance[i]
                # line1 = "%.2f" % (instance[i])
                # line = line + float(line1)
                # print float(instance[i])
                line += float("%.2f" % instance[i])
            print (line)

######################################################################
# Test code - Cosine
######################################################################
dataset = loadCSV(r"C:\Users\cinti\Downloads\iris_original.csv")
clustering = kmeans(dataset, 3)
print("\nSum of Squared Error is: "+str(clustering["withinss"]))

a=clustering["withinss"]
import csv
with open("Kmeans_cosine_SSE.csv", "w") as fp_out:
    print(clustering["withinss"],file=fp_out)
    
clust = repeatedKMeans(dataset, 3, 100)

[(6.3, 3.3, 6.0, 2.5), (7.1, 3.0, 5.9, 2.1), (6.3, 2.9, 5.6, 1.8), (6.5, 3.0, 5.8, 2.2), (7.3, 2.9, 6.3, 1.8), (6.7, 2.5, 5.8, 1.8), (6.4, 2.7, 5.3, 1.9), (6.8, 3.0, 5.5, 2.1), (5.8, 2.8, 5.1, 2.4), (6.4, 3.2, 5.3, 2.3), (6.5, 3.0, 5.5, 1.8), (7.7, 2.6, 6.9, 2.3), (6.9, 3.2, 5.7, 2.3), (6.7, 3.3, 5.7, 2.1), (7.2, 3.2, 6.0, 1.8), (6.4, 2.8, 5.6, 2.1), (7.2, 3.0, 5.8, 1.6), (7.4, 2.8, 6.1, 1.9), (6.4, 2.8, 5.6, 2.2), (6.1, 2.6, 5.6, 1.4), (7.7, 3.0, 6.1, 2.3), (6.3, 3.4, 5.6, 2.4), (6.4, 3.1, 5.5, 1.8), (6.9, 3.1, 5.4, 2.1), (6.7, 3.1, 5.6, 2.4), (6.9, 3.1, 5.1, 2.3), (6.8, 3.2, 5.9, 2.3), (6.7, 3.3, 5.7, 2.5), (6.7, 3.0, 5.2, 2.3), (6.5, 3.0, 5.2, 2.0), (6.2, 3.4, 5.4, 2.3)]
[(7.0, 3.2, 4.7, 1.4), (6.4, 3.2, 4.5, 1.5), (6.9, 3.1, 4.9, 1.5), (6.5, 2.8, 4.6, 1.5), (5.7, 2.8, 4.5, 1.3), (6.3, 3.3, 4.7, 1.6), (6.6, 2.9, 4.6, 1.3), (6.1, 2.9, 4.7, 1.4), (5.6, 3.0, 4.5, 1.5), (6.2, 2.2, 4.5, 1.5), (5.9, 3.2, 4.8, 1.8), (6.3, 2.5, 4.9, 1.5), (6.1, 2.8, 4.7, 1.2), (6.8, 2.8, 4.8, 1.4), (6.7, 3.